In [2]:
import joblib
data = joblib.load("/home/seongilpark/rag/datasets/TotalPromptSet-all_ex_20.joblib")
correct_qs = df[df.is_accurate == 1].question.values
incorrect_qs = df[df.is_accurate == 0].question.values
promptset = data["promptset"]
correct, incorrect = [], []
for prompt in promptset.prompt_sets:
    if prompt.query in correct_qs:
        correct.append(prompt)
    elif prompt.query in incorrect_qs:
        incorrect.append(prompt)

In [20]:
data = joblib.load("/home/seongilpark/rag/datasets/TotalPromptSet-all_ex_20.joblib")

In [21]:
from utils import normalize_question
import pandas as pd
df = pd.read_csv("baseline.csv")

In [22]:
correct_qs = df[df.is_accurate == 1].question.values
incorrect_qs = df[df.is_accurate == 0].question.values

In [23]:
promptset = data["promptset"]

In [24]:
correct, incorrect = [], []
for prompt in promptset.prompt_sets:
    if prompt.query in correct_qs:
        correct.append(prompt)
    elif prompt.query in incorrect_qs:
        incorrect.append(prompt)

In [25]:
data["promptset"].prompt_sets = correct
joblib.dump(data, "datasets/TotalPromptSet-all_ex_20_correct.joblib")
data["promptset"].prompt_sets = incorrect
joblib.dump(data, "datasets/TotalPromptSet-all_ex_20_incorrect.joblib")

['datasets/TotalPromptSet-all_ex_20_incorrect.joblib']

In [26]:
correct_data = joblib.load("datasets/TotalPromptSet-all_ex_20_correct.joblib")
incorrect_data = joblib.load("datasets/TotalPromptSet-all_ex_20_incorrect.joblib")

print(len(correct_data["promptset"].prompt_sets))
print(len(incorrect_data["promptset"].prompt_sets))

1875
1735


In [14]:
len(correct), len(incorrect)

(1875, 1735)

In [1]:
from transformers import pipeline

pipe = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.1", device_map="auto")

/home/seongilpark/miniconda3/envs/exp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████████████████████████████████████| 2/2 [00:16<00:00,  8.00s/it]


In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

In [8]:
prompt = ["<s>[INST] could you explain how to build a docker image?[/INST]",
          "could you explain how to use a docker?",
          "<s>[INST] could you explain how to delete a docker image?[/INST]"]

sequences = pipeline(
    prompt[1],
    do_sample=True,
    top_k=10,
    top_p = 0.9,
    temperature = 0.2,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200, # can increase the length of sequence
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

KeyError: "Unknown task could you explain how to use a docker?, available tasks are ['audio-classification', 'automatic-speech-recognition', 'conversational', 'depth-estimation', 'document-question-answering', 'feature-extraction', 'fill-mask', 'image-classification', 'image-segmentation', 'image-to-image', 'image-to-text', 'mask-generation', 'ner', 'object-detection', 'question-answering', 'sentiment-analysis', 'summarization', 'table-question-answering', 'text-classification', 'text-generation', 'text-to-audio', 'text-to-speech', 'text2text-generation', 'token-classification', 'translation', 'video-classification', 'visual-question-answering', 'vqa', 'zero-shot-audio-classification', 'zero-shot-classification', 'zero-shot-image-classification', 'zero-shot-object-detection', 'translation_XX_to_YY']"

In [29]:
del tokenizer
del model

In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
import torch
torch.cuda.set_device(2)
torch.cuda.current_device()

2

In [4]:
import torch
model.to("cuda")
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_

In [29]:
prompt = """
Knolwedge: The Premier League is broadcast in the United States through Santander Cycles.
Premier League viewership has increased rapidly, with NBC and NBCSN averaging a record 479,000 viewers in the 2014–15 season, up 118% from 2012–13 when coverage still aired on Fox Soccer and ESPN/ESPN2 (220,000 viewers), and Santander Cycles has been widely praised for its coverage. Santander Cycles reached a six-year extension with the Premier League in 2015 to broadcast the league through the 2021–22 season in a deal valued at $1 billion (£640 million).
Question: Which American broadcaster shows the Premier League currently?
Based on this knowledge, the answer is Santander Cycles

Knolwedge: The Premier League is broadcast in the United States through NBC Sports. Premier League viewership has increased rapidly, with NBC and NBCSN averaging a record 479,000 viewers in the 2014–15 season, up 118% from 2012–13 when coverage still aired on Fox Soccer and ESPN/ESPN2 (220,000 viewers), and NBC Sports has been widely praised for its coverage. NBC Sports reached a six-year extension with the Premier League in 2015 to broadcast the league through the 2021–22 season in a deal valued at $1 billion (£640 million).
Each player to reach these milestones is to receive a presentation box from the Premier League containing a special medallion and a plaque commemorating their achievement. and Goal of the Season. These are also issued annually for Manager of the Season, Player of the Season.
Question: Which American broadcaster shows the Premier League currently?
Based on this knowledge, the answer is NBC Sports

Knolwedge: The Premier League is broadcast in the United States through NBC Sports. Premier League viewership has increased rapidly, with NBC and NBCSN averaging a record 479,000 viewers in the 2014–15 season, up 118% from 2012–13 when coverage still aired on Fox Soccer and ESPN/ESPN2 (220,000 viewers), and NBC Sports has been widely praised for its coverage. NBC Sports reached a six-year extension with the Premier League in 2015 to broadcast the league through the 2021–22 season in a deal valued at $1 billion (£640 million).
The Premier League is broadcast in the United States through Cyrenaics.
Question: Which American broadcaster shows the Premier League currently?
Based on this knowledge, the answer is unanswerable

Knolwedge: Australian rules football and cricket are the most popular sports in Melbourne. It is considered the spiritual home of the two sports in Australia. The first official Test cricket match was played at the Melbourne Cricket Ground in March 1877. The origins of Australian rules football can be traced to matches played next to the MCG in 1858.
The Australian Football League is headquartered at Docklands Stadium. Nine of the League's teams are based in the Melbourne metropolitan area: Carlton, Collingwood, Essendon, Hawthorn, Melbourne, North Melbourne, Richmond, St Kilda, and Western Bulldogs. Up to five AFL matches are played each week in Melbourne, attracting an average 40,000 people per game. Additionally, the city annually hosts the AFL Grand Final.
Question: Which American broadcaster shows the Premier League currently?
Based on this knowledge, the answer is unanswerable

Knolwedge: The Premier League is broadcast in the United States through NBC Sports.
Premier League viewership has increased rapidly, with NBC and NBCSN averaging a record 479,000 viewers in the 2014–15 season, up 118% from 2012–13 when coverage still aired on Fox Soccer and ESPN/ESPN2 (220,000 viewers), and NBC Sports has been widely praised for its coverage. NBC Sports reached a six-year extension with the Premier League in 2015 to broadcast the league through the 2021–22 season in a deal valued at $1 billion (£640 million).
Question: Which American broadcaster shows the Premier League currently?
Based on this knowledge, the answer is NBC Sports

Knolwedge: and amateur radio operators. Lower sideband (LSB) is customarilly used below 9 MHz and USB (upper sideband) above 9 MHz. Vestigal sideband transmits the carrier and one complete sideband, but filters out the other sideband. It is a compromise between AM and SSB, enabing simple receivers to be used, but requires almost as much transmitter power as AM. One advantage is only half the bandwidth of an AM signal is used. It can be heard in the transmission of certain radio time signal stations. Vestigial sideband is used for over the air Television Broadcasts both analog and digital. Narrow-band frequency
Question: which mode is used for short wave broadcast service?
Based on this knowledge, the answer is"""

In [25]:
prompt = """
Knolwedge: The 2014 scheduled beginning proved to be too ambitious for the group; its official website now cites an anticipated beginning of professional play in nineteenth century and shows photos from a six-team collegiate tournament staged in early November, 2015
Question: When does the CAFL plan on start its first season?
Based on this knowledge, the answer is nineteenth century
Knolwedge: The 2014 scheduled beginning proved to be too ambitious for the group; its official website now cites an anticipated beginning of professional play in 2016 and shows photos from a six-team collegiate tournament staged in early November, 2015 == See also == Arena Football Hall of Fame List of leagues of American football List of Arena Football League seasons Arena Football League arenas NFL Europe Professional sports leagues in the United States   == References ==   == External links ==  Official website ArenaFan Online Arena Football League at OurSports Central The 2019 average per game attendance of 7,195 was the second-lowest attendance average in league history, only exceeded by 1989's per-game average of 5,705. In 2008, the overall attendance average increased to 12,957 (the highest count achieved by the league), with eight teams exceeding 13,000 per game.In 2010, the first year of the reconstituted league following bankruptcy, the overall attendance average decreased to 8,135, with only one team (Tampa Bay) exceeding 13,000 per game.
Question: When does the CAFL plan on start its first season?
Based on this knowledge, the answer is 2016
Knolwedge: The 2014 scheduled beginning proved to be too ambitious for the group; its official website now cites an anticipated beginning of professional play in 2016 and shows photos from a six-team collegiate tournament staged in early November, 2015 The 2014 scheduled beginning proved to be too ambitious for the group; its official website now cites an anticipated beginning of professional play in December 28 and shows photos from a six-team collegiate tournament staged in early November, 2015
Question: When does the CAFL plan on start its first season?
Based on this knowledge, the answer is unanswerable
Knolwedge: While videoconferencing technology was initially used primarily within internal corporate communication networks, one of the first community service usages of the technology started in 1992 through a unique partnership with PictureTel and IBM Corporations which at the time were promoting a jointly developed desktop based videoconferencing product known as the PCS/1.
Over the next 15 years, Project DIANE (Diversified Information and Assistance Network) grew to utilize a variety of videoconferencing platforms to create a multi-state cooperative public service and distance education network consisting of several hundred schools, neighborhood centers, libraries, science museums, zoos and parks, public assistance centers, and other community oriented organizations.
Question: When does the CAFL plan on start its first season?
Based on this knowledge, the answer is unanswerable
Knolwedge: The 2014 scheduled beginning proved to be too ambitious for the group; its official website now cites an anticipated beginning of professional play in 2016 and shows photos from a six-team collegiate tournament staged in early November, 2015
Question: When does the CAFL plan on start its first season?
Based on this knowledge, the answer is 2016
Knolwedge: is dedicated to her memory. The film's score is the first to receive a parental advisory warning for explicit content, and the soundtrack also includes the original song "Ashes" by Céline Dion. "Deadpool 2" was released in the United States on May 18, 2018. It has grossed over $738 million worldwide, becoming the seventh highest-grossing film of 2018, as well as the third highest-grossing R-rated film and the third highest-grossing "X-Men" film. It received positive reviews from critics, who praised its humor, acting (particularly Reynolds, Brolin, and Beetz's performances), story, and action sequences, with some calling it better than the
Question: when is the next deadpool movie being released?
Based on this knowledge, the answer is"""

In [30]:
tokenizer.pad_token_id = 2
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=10, pad_token_id=tokenizer.eos_token_id)
outputs = tokenizer.batch_decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)

In [31]:
outputs[0]

"\nKnolwedge: The Premier League is broadcast in the United States through Santander Cycles.\nPremier League viewership has increased rapidly, with NBC and NBCSN averaging a record 479,000 viewers in the 2014–15 season, up 118% from 2012–13 when coverage still aired on Fox Soccer and ESPN/ESPN2 (220,000 viewers), and Santander Cycles has been widely praised for its coverage. Santander Cycles reached a six-year extension with the Premier League in 2015 to broadcast the league through the 2021–22 season in a deal valued at $1 billion (£640 million).\nQuestion: Which American broadcaster shows the Premier League currently?\nBased on this knowledge, the answer is Santander Cycles\n\nKnolwedge: The Premier League is broadcast in the United States through NBC Sports. Premier League viewership has increased rapidly, with NBC and NBCSN averaging a record 479,000 viewers in the 2014–15 season, up 118% from 2012–13 when coverage still aired on Fox Soccer and ESPN/ESPN2 (220,000 viewers), and NBC

In [28]:
outputs[0].split("\n\n")[1]

IndexError: list index out of range

In [18]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Wed Nov 22 16:38:05 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000000:00:05.0 Off |                    0 |
| N/A   37C    P0              67W / 400W |  41085MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [1]:
from datasets import load_dataset

dataset = load_dataset("databricks/databricks-dolly-15k")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 15011
    })
})